# Spatial Distribution of Methane in Nigeria

Downloading Methane data for Nigeria

## Import and authenticate the Earth Engine Python API

Along with the [`geemap`](https://geemap.org/) library.

In [1]:
import ee
import geemap
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from matplotlib.dates import date2num


In [2]:
ee.Authenticate()

# Initialize the Earth Engine API with the project name
ee.Initialize(project='ee-addy')


## Crearing Area of Interest (aoi)
myaoi

In [66]:
# Area of interest (AOI)
countries = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1');
nigeria = countries.filter(ee.Filter.eq('ADM0_NAME', 'Nigeria'));

# Create an instance of geemap
map = geemap.Map()
map.setCenter(6, 8.0, 6.2)
map.addLayer(nigeria, {'color': 'red'}, 'My AOI')
map

Map(center=[8.0, 6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

## Filter Sentinel 5 off Methane Collection

Using the AOI to spatially and temporally filter data.

In [67]:
# spatial and temporal filtering of sentinel 2 collection
methane2021 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4")\
.filterBounds(nigeria)\
  .filterDate('2021-01-01', '2021-12-31')\
  .select('CH4_column_volume_mixing_ratio_dry_air');

In [68]:
# Reduce the collected imagery to mean values
mean_methane = methane2021.reduce(ee.Reducer.mean()).clip(nigeria);
#print(mean_methane)

min_methane = mean_methane.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=nigeria,
    scale=1113.2  # Resolution
)
max_methane = mean_methane.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=nigeria,
    scale=1113.2  # Resolution
)
# Extracting the minimum value
min_methane_value = min_methane.getInfo()
max_methane_value = max_methane.getInfo()

print(min_methane_value)
print(max_methane_value)




KeyboardInterrupt: 

## Visualization

In [69]:
#visualization
vizPram = {
  'min':1779,
  'max':1960,
  'palette': ('black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red')};

m = geemap.Map()
m.add_basemap()
m.add_basemap("SATELLITE")
map.setCenter(6, 8.0, 6.2)

m.addLayer(mean_methane,vizPram, 'methane2021');
m.addLayer(nigeria, {'color': 'FF0000'}, 'My Region of Interest')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export image

In [70]:
# Exporting data to GEE Asset
exportImg = True

if exportImg:
    # Exporting image to Google Drive
    export_params = {
        'image': mean_methane,
        'scale': 1113.2,  # Pixel size
        'region': nigeria,
        'maxPixels': 1e13
    }
    ee.batch.Export.image.toDrive(**export_params).start()
else:
    pass